In [2]:
def loadRules(filePath):
  rules_list=[]
  with open(filePath,'r') as file:
    for line in file:
      line= file.readlines()
      if not line:continue
      for rule_line in line:
            rule_line = rule_line.strip()
            if not rule_line:continue

            if 'THEN' in rule_line:
                premise_part, conclusion_part = rule_line.split('THEN')
                premises_cleans = premise_part.replace('IF', '').strip()
                if 'OR' in premises_cleans:
                  or_condition = [cod.strip() for cod in premises_cleans.split('OR')]
                  for cod in or_condition:
                    rules_list.append({'IF': [cod], 'THEN': conclusion_part})
                else:
                  and_condition = [cod.strip() for cod in premises_cleans.split('AND')]
                  rules_list.append({'IF': and_condition ,'THEN': conclusion_part})

    return rules_list

In [3]:
def loadFacts(filePath):
  facts=[]
  with open(filePath,'r') as file:
    for line in file:
      line=line.strip()
      if not line:continue
      facts.append(line)
  return facts

In [4]:
path = 'rules.txt'
rules = loadRules(path)
for rule in rules:
  print(rule)

{'IF': ['shape is round', 'color is red', 'size is medium'], 'THEN': ' fruit is apple'}
{'IF': ['shape is round', 'color is red', 'size is small'], 'THEN': ' fruit is cherry'}
{'IF': ['skin_smell'], 'THEN': ' perfumed'}
{'IF': ['fruit is lemon'], 'THEN': ' citrus_fruit'}
{'IF': ['fruit is orange'], 'THEN': ' citrus_fruit'}
{'IF': ['fruit is pomelo'], 'THEN': ' citrus_fruit'}
{'IF': ['fruit is grapefruit'], 'THEN': ' citrus_fruit'}
{'IF': ['size is medium', 'color is yellow', 'perfumed'], 'THEN': ' fruit is lemon'}
{'IF': ['size is medium', 'color is green'], 'THEN': ' fruit is kiwi'}
{'IF': ['size is big', 'perfumed', 'color is orange', 'citrus_fruit'], 'THEN': ' fruit is grapefruit'}
{'IF': ['perfumed', 'color is orange', 'size is medium'], 'THEN': ' fruit is orange'}
{'IF': ['perfumed', 'color is red', 'size is small', 'seeds = 0'], 'THEN': ' fruit is strawberry'}
{'IF': ['diameter < 2'], 'THEN': ' size is small'}
{'IF': ['diameter > 10'], 'THEN': ' size is big'}
{'IF': ['diameter > 

In [5]:
path = 'Facts.txt'
facts = loadFacts(path)
for fact in facts:
  print(fact)

seeds = 0
diameter = 7
skin_smell
color is orange


In [6]:
goal = 'citrus_fruit'

In [7]:
rules

[{'IF': ['shape is round', 'color is red', 'size is medium'],
  'THEN': ' fruit is apple'},
 {'IF': ['shape is round', 'color is red', 'size is small'],
  'THEN': ' fruit is cherry'},
 {'IF': ['skin_smell'], 'THEN': ' perfumed'},
 {'IF': ['fruit is lemon'], 'THEN': ' citrus_fruit'},
 {'IF': ['fruit is orange'], 'THEN': ' citrus_fruit'},
 {'IF': ['fruit is pomelo'], 'THEN': ' citrus_fruit'},
 {'IF': ['fruit is grapefruit'], 'THEN': ' citrus_fruit'},
 {'IF': ['size is medium', 'color is yellow', 'perfumed'],
  'THEN': ' fruit is lemon'},
 {'IF': ['size is medium', 'color is green'], 'THEN': ' fruit is kiwi'},
 {'IF': ['size is big', 'perfumed', 'color is orange', 'citrus_fruit'],
  'THEN': ' fruit is grapefruit'},
 {'IF': ['perfumed', 'color is orange', 'size is medium'],
  'THEN': ' fruit is orange'},
 {'IF': ['perfumed', 'color is red', 'size is small', 'seeds = 0'],
  'THEN': ' fruit is strawberry'},
 {'IF': ['diameter < 2'], 'THEN': ' size is small'},
 {'IF': ['diameter > 10'], 'THEN

In [8]:
facts

['seeds = 0', 'diameter = 7', 'skin_smell', 'color is orange']

In [9]:
def evaluate_numeric_condition(condition, facts):
    """Evaluate numeric conditions like 'diameter > 2' against facts like 'diameter = 7'"""
    operators = ['>', '<', '>=', '<=', '=', '!=']
    for op in operators:
        if op in condition:
            parts = condition.split(op)
            if len(parts) == 2:
                var_name = parts[0].strip() # diameter
                threshold = float(parts[1].strip()) # 2
                for ope in operators:
                    for fact in facts:
                        if var_name in fact and ope in fact:
                            fact_parts = fact.split(ope)
                            if len(fact_parts) == 2 and fact_parts[0].strip() == var_name:
                                try:
                                    value = float(fact_parts[1].strip())
                                    if op == '>' and value > threshold:
                                        return True, condition
                                    elif op == '<' and value < threshold:
                                        return True, condition
                                    elif op == '>=' and value >= threshold:
                                        return True, condition
                                    elif op == '<=' and value <= threshold:
                                        return True, condition
                                    elif op == '=' and value == threshold:
                                        return True, condition
                                    elif op == '!=' and value != threshold:
                                        return True, condition
                                    return False, None
                                except ValueError:
                                    pass

    return False, None


In [10]:
def add_fact_uniquely(facts, fact):
    """Add a fact to the facts list only if it's not already there"""
    if fact not in facts:
        facts.append(fact)
        return True
    return False

In [11]:
def backward_chaining_with_trace(rules, facts, goal, proven_facts=None):
    if proven_facts is None:
        proven_facts = set()
    print(f"Trying to prove: {goal}")
    if goal in facts:
        print(f"✓ Goal {goal} already in facts")
        return True
    is_numeric = any(op in goal for op in ['>', '<', '>=', '<=', '=', '!='])
    if is_numeric:
        numeric_result, condition = evaluate_numeric_condition(goal, facts)
        if numeric_result:
            add_fact_uniquely(facts, condition)
            print(f"✓ Numeric goal {goal} satisfied by facts")
            return True

    relevant_rules = [rule for rule in rules if rule['THEN'] == goal]
    print(f"Found {len(relevant_rules)} rules that can conclude {goal}")

    if not relevant_rules:
        print(f"✗ No rules can conclude {goal}")
        return False

    for i, rule in enumerate(relevant_rules):
        print(f"Trying rule {i+1}: IF {rule['IF']} THEN {rule['THEN']}")

        all_conditions_satisfied = True
        for condition in rule['IF']:
            if condition in facts:
                print(f"  ✓ Condition {condition} already satisfied")
                continue
            is_numeric_condition = any(op in condition for op in ['>', '<', '>=', '<=', '=', '!='])
            if is_numeric_condition:
                numeric_result, satisfied_condition = evaluate_numeric_condition(condition, facts)
                if numeric_result:
                    add_fact_uniquely(facts, satisfied_condition)
                    print(f"  ✓ Numeric condition {condition} satisfied by facts")
                    continue
            if condition in proven_facts:
                print(f"  ✓ Condition {condition} was proven earlier")
                continue
            print(f"  Need to prove sub-goal: {condition}")
            if backward_chaining_with_trace(rules, facts, condition,proven_facts):
                add_fact_uniquely(facts, condition)
                proven_facts.add(condition)
                print(f"  ✓ Sub-goal {condition} proven and added to facts")
            else:
                all_conditions_satisfied = False
                print(f"  ✗ Failed to prove {condition}")
                break

        if all_conditions_satisfied:
            add_fact_uniquely(facts, goal)
            proven_facts.add(goal)
            print(f"✓ All conditions met, goal {goal} proven")
            return True

    print(f"✗ All rules failed, cannot prove {goal}")
    return False


In [12]:
for rule in rules:
    rule['THEN'] = rule['THEN'].strip()

facts_copy = facts.copy()
proven_facts_set = set()
result = backward_chaining_with_trace(rules, facts_copy, goal, proven_facts=proven_facts_set)
print(f"\nGoal '{goal}' can be achieved: {result}")
print("Final facts:", facts_copy)


Trying to prove: citrus_fruit
Found 4 rules that can conclude citrus_fruit
Trying rule 1: IF ['fruit is lemon'] THEN citrus_fruit
  Need to prove sub-goal: fruit is lemon
Trying to prove: fruit is lemon
Found 1 rules that can conclude fruit is lemon
Trying rule 1: IF ['size is medium', 'color is yellow', 'perfumed'] THEN fruit is lemon
  Need to prove sub-goal: size is medium
Trying to prove: size is medium
Found 1 rules that can conclude size is medium
Trying rule 1: IF ['diameter > 2', 'diameter < 10'] THEN size is medium
  ✓ Numeric condition diameter > 2 satisfied by facts
  ✓ Numeric condition diameter < 10 satisfied by facts
✓ All conditions met, goal size is medium proven
  ✓ Sub-goal size is medium proven and added to facts
  Need to prove sub-goal: color is yellow
Trying to prove: color is yellow
Found 0 rules that can conclude color is yellow
✗ No rules can conclude color is yellow
  ✗ Failed to prove color is yellow
✗ All rules failed, cannot prove fruit is lemon
  ✗ Failed 

In [40]:
def is_condition_met(condition, facts):
    """Check if a symbolic or numeric condition is met"""
    if any(op in condition for op in ['>', '<', '>=', '<=', '=', '!=']):
        return evaluate_numeric_condition(condition, facts)[0]
    else:
        return condition in facts




In [49]:
def forward_chaining(rules, facts, goal):
    new_inferred = True
    fired_rules = []
    fact_sources = []

    while new_inferred:
        new_inferred = False

        for i, rule in enumerate(rules):
            condition_met = all(is_condition_met(cond, facts) for cond in rule['IF'])

            if condition_met:
                conclusion = rule['THEN'].strip()


                if conclusion not in facts:
                    print(f"Inferred: {conclusion} (by Rule {i+1}: IF {rule['IF']} THEN {conclusion})")
                    fired_rules.append(f"Rule {i+1}: IF {rule['IF']} THEN {conclusion}")
                    fact_sources.append((conclusion, f"Rule {i+1}: IF {rule['IF']} THEN {conclusion}"))
                    add_fact_uniquely(facts, conclusion)
                    new_inferred = True

                elif f"Rule {i+1}: IF {rule['IF']} THEN {conclusion}" not in fired_rules:

                    fired_rules.append(f"Rule {i+1}: IF {rule['IF']} THEN {conclusion}")
                    fact_sources.append((conclusion, f"Rule {i+1}: IF {rule['IF']} THEN {conclusion}"))


    return goal in facts, facts, fired_rules, fact_sources


In [50]:
for rule in rules:
    rule['THEN'] = rule['THEN'].strip()

facts_copy = facts.copy()


result, final_facts, used_rules, fact_sources = forward_chaining(rules, facts_copy, goal)

print(f"\n Goal: '{goal}' can be achieved: {result}")
print("\n Rules used:")

for fact, rule in fact_sources:
    print(f"- {rule} (Fact: {fact})")

print("\n Final facts:")
for f in final_facts:
    print("-", f)



 Goal: 'citrus_fruit' can be achieved: True

 Rules used:
- Rule 3: IF ['skin_smell'] THEN perfumed (Fact: perfumed)
- Rule 5: IF ['fruit is orange'] THEN citrus_fruit (Fact: citrus_fruit)
- Rule 11: IF ['perfumed', 'color is orange', 'size is medium'] THEN fruit is orange (Fact: fruit is orange)
- Rule 15: IF ['diameter > 2', 'diameter < 10'] THEN size is medium (Fact: size is medium)

 Final facts:
- seeds = 0
- diameter = 7
- skin_smell
- color is orange
- perfumed
- size is medium
- fruit is orange
- citrus_fruit
